<a href="https://colab.research.google.com/github/minhajabedin217/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_swnlzv.png" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div>

<br><br><br><br>

# Simulation of dose-response curves of agonists using affinity values

In this tutorial, we will simulate a mathematical model of a signaling pathway to obtain dose-response curves of agonists, from which the *potency (EC<sub>50</sub>)* values of agonists can be inferred.


This tutorial, along with the others in this series, can be completed in Google Colab. If you'd like to open this notebook in Colab, you can use the following link:

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial1.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>

</div>  


## Installing the SSB toolkit library

In [1]:
%%capture
!pip install --quiet ssbtoolkit

If you are working within the **Google Colaboratory** environment, please execute the subsequent cell to mitigate potential version compatibility issues. If not, proceed directly to the next step.

In [ ]:
import os
os.kill(os.getpid(), 9)

### Importing dependencies

In [1]:
import ssbtoolkit as ssb
import numpy as np
import pandas as pd

AttributeError: 'FileFinder' object has no attribute 'find_module'

In [3]:
# Check if ssbtoolkit is imported by trying to access a utility function
try:
    ssb.Utils
    print("ssbtoolkit imported successfully.")
except NameError:
    print("ssbtoolkit not imported. Please ensure cell e344d8de was run successfully.")

ssbtoolkit not imported. Please ensure cell e344d8de was run successfully.


# How to run ensure cell e344d8de was run successfully.

In [5]:
import ssbtoolkit as ssb
import numpy as np
import pandas as pd

# Check if ssbtoolkit is imported by trying to access a utility function
try:
    ssb.Utils
    print("ssbtoolkit imported successfully.")
except NameError:
    print("ssbtoolkit not imported. Please ensure the import cell was run successfully.")

AttributeError: 'FileFinder' object has no attribute 'find_module'

If you are using the **Ebrains Platform**, execute the following cell to set up the environment. If you are not using **Ebrains**, skip the following step and proceed with the tutorial.

In [4]:
import os
os.environ['BNGPATH'] = '/opt/app-root/src/.local/lib/python3.8/site-packages/bionetgen/bng-linux/'

## Loading experimental data

Once the SSBtoolkit environment is set up, we are ready to start the simulation.

We will begin by loading the affinity data for several selective agonists of the Adenosine 2A receptor. This experimental affinity data was obtained from *[Varani, K. et al., 2015](https://doi.org/10.1021/acs.jmedchem.5b00215)* be directly downloaded from the repository with the following command:

In [3]:
!curl -s -O https://raw.githubusercontent.com/rribeiro-sci/SSBtoolkit/main/data/A2AR_binding_data.csv

In [4]:
data = pd.read_csv('A2AR_binding_data.csv')
data

NameError: name 'pd' is not defined

## Affinity Value Conversion

Experimental affinity values often come in different "flavors."

The SSB framework accepts affinity values only in the form of  pK<sub>d</sub>. Since the affinity values in our dataset are reported as K<sub>i</sub> and EC<sub>50</sub>, we need to convert them to pK<sub>i</sub> and pEC<sub>50</sub>, respectively.

In [5]:
data['pKi'] = data.Ki.apply(lambda x: -np.log10(x*1E-6))
data['pEC50']=data.EC50.apply(lambda x: -np.log10(x*1E-6))
data

NameError: name 'data' is not defined

## Selection of the signaling pathway

The core of the SSB framework is, naturally, the mathematical models of GPCR signaling pathways.

G-protein sub-families are classified by their $\alpha$ subunits, which are used to identify their corresponding signaling pathways:

* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 For more details, refer to the [The SSB toolkit](https://ssbtoolkit.readthedocs.io/) documentation.

We can manually define the G$\alpha$ pathway you want to work with, or use the SSB Toolkit’s built-in function `Utils.GetGProtein()` to query our internal database of human GPCR pathways using the UNIPROT ID of the receptor. The UNIPROT ID for the human Adenosine A2 receptor is [P29274](https://www.uniprot.org/uniprot/P29274).



In [ ]:
uniprotID = 'P29274'
gprotein = ssb.Utils.GetGProtein(uniprotID)
gprotein

<span style="color:black">⚠️ WARNING: The toolkit is specifically designed for human GPCRs. Querying pathways for GPCRs other than human may be included in future updates. However, if you wish to study a non-human GPCR, you can still use the SSB Toolkit by manually setting the G$\alpha$ pathway.</span>


##  Preparation, Simulation and Analysis

To obtain a dose-response curve from the simulation of signaling pathways, we need to perform individual simulations of the pathway across a series of ligand concentrations, similar to what would be done in a wet lab.

We will define an array of ligand concentrations using a geometric progression. This approach is chosen because it addresses the impact of dilution fractions on the accuracy of experimentally estimated K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values. The spacing between adjacent concentration values affects the precision of the measurements, particularly in the linear portion of the curve. By using a geometric progression, we can better simulate experimental conditions where each concentration is a power of 2 of the previous concentration *[Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426)*

<span style="color:black"> ⚠️ WARNING: The SSB toolkit uses μM as default unit for concentration values.</span>


In [ ]:
# setting the range of ligand concentrations
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

SSB simulations are also sensitive to the concentration of the protein, which must be expressed in <b><i>μM</i></b>. However, functional assays typically report protein concentrations in <b><i>μg</i></b>.

The experimental data we are using was obtained from a functional assay with 50 μg of protein. To convert this protein concentration from <b><i>μg</i></b> to <b><i>μM</i></b>, you can use the SSB Toolkit's built-in function `Utils.MicrogramsToNanomolar()`.


In [ ]:
# the following function takes as input the UniProtID and the concentration in μg.
prot_conc = ssb.Utils.MicrogramsToNanomolar(uniprotID, 50)
print(prot_conc, 'μM')

Finally, we need to define the length and number of time steps (i.e., the resolution) for the simulation.

In [ ]:
time = 10000   # time of simulation in seconds
nsteps = 1000  # number of time steps

## Integration of ODEs

Once all the simulation parameters have been defined, we are ready to proceed with the simulations. Simulating a mathematical model of a signaling pathway involves integrating a set of ordinary differential equations (ODEs) over time. Each ODE represents a molecular event within the signaling pathway, and integrating these equations describes how the concentration of species in the model changes over time.

A key aspect of this tutorial is using the drug-receptor affinity value (K<sub>d</sub>) to initiate the model. With K<sub>d</sub> values, we can calculate the fraction of receptors occupied by the ligand at equilibrium. According to *occupancy theory*, this fraction represents the concentration of activated receptors at equilibrium *[Kenakin T., 2004 ](https://doi.org/10.1016/j.tips.2004.02.012)*. 📖 For more details, refer to the [Docs](https://ssbtoolkit.readthedocs.io/).

In this tutorial, we will simulate dose-response curves of agonists targeting the Adenosine A2 receptor using the SSB Toolkit’s built-in class `Simulation.Activation()`.

<span style='color:black'>ℹ️ If you are interested in studying antagonists, follow the tutorial on [Simulation of dose-responses curves of antagonists](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial2.ipynb).</span>

In [ ]:
# we will start by creating a simulation instance.
sim = ssb.Simulation.Activation()

In [ ]:
# configuration of simulation parameters
sim.SetSimulationParameters(ligands=data.id.to_list()[:2],
                            affinities=data.pKi.to_list()[:2],
                            pathway=gprotein,
                            receptor_conc=prot_conc,
                            lig_conc_range=lig_conc_range,
                            ttotal=time,
                            nsteps=nsteps,
                            binding_kinetics=False)

In [ ]:
# Starting the simulation
sim.Run()

At the end of the simulation, the concentration values of the species within the signaling pathway will be saved in the instance.

The response of a signaling pathway is typically represented by the increase or decrease in the concentration of one of the species described by the model. To predict the dose-response curve, we need to first extract the maximum concentration value observed for a particular species from each individual simulation (for each ligand concentration). These maximum values will then be fitted to a logistic regression.

To achieve this, we will use the following analysis function:

In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.ShowCurve()

Finnaly, from the dose-response curves, we can interpolate the EC<sub>50</sub> values.

In [ ]:
sim.ShowPotency()

💡 The predicted potency values can be exported as a Python dictionary using the function `sim.PotencyToDict()`, or saved to a CSV file using `sim.PotencyToCSV()`.


## Congratulations!

Congratulations on completing this tutorial notebook! If you enjoyed working through it and would like to continue exploring the SSB Toolkit, we encourage you to complete the rest of the tutorials in this series.

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{ribeiro_ssb_2022,
    title={{SSB} toolkit: from molecular structure to subcellular signaling pathways.},
    author={Ribeiro, Rui Pedro and Gossen, Jonas and Rossetti, Giulia and Giorgetti, Alejandro},
    publisher={bioRxiv},
    url={https://www.biorxiv.org/content/10.1101/2022.11.08.515595v1},
    doi={10.1101/2022.11.08.515595},
    year={2022}
}
```



## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  